Imports
=======

In [1]:
import pandas as pd
import numpy as np
from random import shuffle
from sklearn import tree
from copy import deepcopy
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import accuracy_score as acc
from sklearn.ensemble import BaggingRegressor
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import cross_validation
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize_scalar
from copy import deepcopy
%matplotlib inline

Подготовка данных
=================
Делим выборку из *train.csv* на train и test в отношении 1:9

In [6]:
# data = pd.read_csv('blogData_train.csv', header=None)
data = pd.read_csv('train.csv', sep=';', index_col='ID')
data.shape

(40000, 73)

In [7]:
# Посмотрим на данные
data.head()

,LOCATION_ID,MODEL_ID,MODEL_NAME,MODEL_OS,MODEL_TYPE,MODEL_BRAND,MODEL_COST_AVG,STORE_REGION,STORE_CITY,STORE_OPEN_DTTM,...,SPECTR_NONSMART_other_brands,SPECTR_NONSMART_samsung,SPECTR_NONSMART_mts,SPECTR_NONSMART_BIG,SPECTR_NONSMART_MEDIUM,SPECTR_NONSMART_SMALL,SPECTR_NONSMART_HD,SPECTR_NONSMART_LOW,SPECTR_NONSMART_GOOD,DEMAND
ID,,,,,,,,,,,,,,,,,,,,,
0,183783,3113,sony xperia e3 lte,gsm-and,смартфон,sony,6797.28,северо-запад,санкт-петербург г,07JUL11:00:00:00,...,0.79,1.00,0.0,0.0,0.86,0.89,0.0,0.87,0.0,0.032258
1,2022103615,480949,stark impress strong lte dual sim,gsm-and,смартфон,stark,6432.20,северо-запад,пушкин г,14APR16:00:00:00,...,0.48,1.00,NaN,0.0,0.40,0.86,0.0,0.62,NaN,0.000100
2,29792,1896,fly iq4490i,gsm-and,смартфон,fly,1885.04,поволжье,альметьевск г,29AUG09:00:00:00,...,0.63,0.70,0.0,0.0,0.65,0.67,0.0,0.66,0.0,0.048790
3,28438,264322,texet tm-224,gsm без ос,не смартфон,texet,1144.40,северо-запад,пушкин г,20OCT09:00:00:00,...,0.53,0.50,0.0,0.0,0.48,0.67,0.0,0.57,0.0,0.000100
4,28590,9086,ark benefit a1,gsm-and,смартфон,ark,3235.54,северо-запад,санкт-петербург г,07JUL09:00:00:00,...,0.50,0.73,0.0,0.0,0.62,0.56,0.0,0.60,0.0,0.000100


In [10]:
# Categorical columns
Cathegorical_columns = data.columns[[isinstance(z, str) for z in data.ix[0]]]
Cathegorical_columns

Index([u'MODEL_NAME', u'MODEL_OS', u'MODEL_TYPE', u'MODEL_BRAND',
       u'STORE_REGION', u'STORE_CITY', u'STORE_OPEN_DTTM', u'STORE_FORMAT',
       u'STORE_BUILDING_TYPE', u'STORE_ADDRESS_LINE', u'STORE_GPS',
       u'STORE_PRICE_TYPE', u'ABC_SMART_WHITE', u'ABC_SMART_BLACK',
       u'ABC_SMART_COLORED', u'ABC_SMART_alcatel', u'ABC_SMART_other_brands',
       u'ABC_SMART_samsung', u'ABC_SMART_мтс', u'ABC_SMART_RES_HD',
       u'ABC_SMART_RES_LOW', u'ABC_SMART_DIAG_BIG', u'ABC_SMART_DIAG_MEDIUM',
       u'ABC_NONSMART_WHITE', u'ABC_NONSMART_BLACK', u'ABC_NONSMART_COLORED',
       u'ABC_NONSMART_alcatel', u'ABC_NONSMART_other_brands',
       u'ABC_NONSMART_samsung', u'ABC_NONSMART_RES_LOW',
       u'ABC_NONSMART_DIAG_MEDIUM', u'ABC_NONSMART_DIAG_SMALL'],
      dtype='object')

In [9]:
# # Количество уникальных значений в категориальных признаках
data['fict'] = [1]*data.shape[0]
a = data.groupby('fict')
dist_val = pd.DataFrame([np.int(a[z].apply(lambda x: len(x.unique()))) for z in Cathegorical_columns])
print(dist_val.values.T)
print('Количество уникальных категориальных значений: '+ str(dist_val.sum().values))
print('Удаляем признаки с номерами: \n'+ str(dist_val.ix[dist_val[0]>100].reset_index()))
data.drop('fict',axis = 1, inplace=True)
# list(dist_val)[[0,1]]

[[456   5   2  33   5  36 246  10   3 354 359  20   4   4   4   4   4   4
    4   4   4   4   4   4   4   4   4   4   4   4   4   4]]
Количество уникальных категориальных значений: [1609]
Удаляем признаки с номерами: 
   index    0
0      0  456
1      6  246
2      9  354
3     10  359


In [5]:
# Delete Categorical columns
data.drop(Cathegorical_columns[[7,14,17]], axis = 1, inplace=True)
data = data.fillna(-2)
Cathegorical_columns = Cathegorical_columns.drop(Cathegorical_columns[[7,14,17]])

## Исследование категориальных признаков 

In [21]:
data_copy[['MODEL_TYPE', 'MODEL_BRAND', 'STORE_GPS']].head()

,x10,x18,x21,y
0,e5df3eff9b,634e3cf3ac,513a3e3f36,2
1,fa0b797a92,bb20e1ca06,992153ed65,4
2,152af2cb2f,ee3501282b,5c5025bd0a,3
3,c4dd2197c3,9e166b965d,acfadc5c01,4
4,e071d01df5,527b6ca8cc,0fc56ea1f0,3


In [7]:
# Delete Categorical columns
data.drop(Cathegorical_columns[[7,14,17]], axis = 1, inplace=True)
data = data.fillna(-2)
data.shape
Cathegorical_columns = Cathegorical_columns.drop(Cathegorical_columns[[7,14,17]])

## Предсказание без категориальных признаков
## Вспомогательные функции

In [5]:
def get_numerated_data(df, categorical_columns):
    simple_df = deepcopy(df)
    for column in categorical_columns:
        simple_df[column] = pd.Categorical.from_array(simple_df[column]).labels
    
    return simple_df

def get_binarized_data(df, categorical_columns):
    binary_df = deepcopy(df)
    for column in categorical_columns:
        binary_df = pd.concat([binary_df, pd.get_dummies(binary_df[column], prefix=column, prefix_sep=': ')], axis=1)
        del binary_df[column]
    
    return binary_df

def split_to_numpy(df, y_column_name):
    #print df
    #print y_column_name
    data_x = df.drop(y_column_name, axis=1).values.astype(np.float32)
    data_y = df[y_column_name].values.astype(np.float32)
    column_names = df.drop(y_column_name, axis=1).columns
    
    return data_x, data_y, column_names

def split_train_test(df, test_size=0.5, random_state=42, y_column_name='y'):
    X = np.asarray(df.drop(y_column_name, axis=1))
    y = np.asarray(df[y_column_name]).ravel()
    trainX, testX, trainY, testY = cross_validation.train_test_split(X, y, test_size=test_size, random_state=42)
    return trainX, testX, trainY, testY

### Необходимые функции

In [33]:
def show_plot(n_est, res, title="", ylabel="mse"):
    fig, ax = plt.subplots()
    ax.plot(n_est, [res[x][1] for x in n_est], 'ro')
#     ax.set_xlim((np.min(n_est) - 10, np.max(n_est) + 10))
#     plt.xlabel("n_estimators")
#     plt.ylabel(ylabel)
#     plt.title("Test result")
#     plt.show()
    
#     fig, ax = plt.subplots()
    ax.plot(n_est, [res[x][0] for x in n_est], 'bo')
    ax.set_xlim((np.min(n_est) - 10, np.max(n_est) + 10))
    plt.xlabel("n_estimators")
    plt.ylabel("mse")
    plt.title(title)
    plt.show()

def round_predict(predict, min_val=0, max_val = 6):
    return np.array([min_val if x < min_val else x for x in [max_val if x>max_val else x for x in np.rint(predict)]]).astype(int)

def get_results_baggind_rsm(n_est, max_samples, max_features, verbose=True):
    results = dict()
    for i in n_est:
        reg = BaggingRegressor(n_estimators=i,
                          max_samples=max_samples, max_features=max_features, random_state=1234, n_jobs=-1, verbose=0)
        reg.fit(trainX, trainY)
#         results[i] = (mse(reg.predict(trainX), trainY), mse(reg.predict(testX), testY))
        predict = reg.predict(testX)
        results[i] = (acc(round_predict(reg.predict(trainX)), trainY), acc(round_predict(predict), testY))
        if verbose:
            print("Count of estimators: " + str(i) + ", score: " + str(results[i][1]))
    return results

def get_results_rf(n_est, oob=False, verbose=True):
    results = dict()
    for i in n_est:
        reg = RandomForestRegressor(n_estimators=i, n_jobs=-1, random_state=1234, verbose=0,
                                    oob_score=oob, max_features='sqrt')
        reg.fit(trainX, trainY)
        predict = reg.predict(testX)
        if oob:
            results[i] = (reg.oob_score_, mse(predict, testY))
        else:
#             results[i] = (mse(reg.predict(trainX), trainY), mse(reg.predict(testX), testY))
            results[i] = (acc(round_predict(reg.predict(trainX)), trainY), acc(round_predict(predict), testY))
        if verbose:
            print("Count of estimators: " + str(i) + ", score: " + str(results[i][1]))
    return results

def get_results_erte(n_est, verbose=True):
    results = dict()
    for i in n_est:
        reg = ExtraTreesRegressor(n_estimators=i, n_jobs=-1, random_state=1234, verbose=0)
        reg.fit(trainX, trainY)
#         results[i] = (mse(reg.predict(trainX), trainY), mse(reg.predict(testX), testY))
        predict = reg.predict(testX)
        results[i] = (acc(round_predict(reg.predict(trainX)), trainY), acc(round_predict(predict), testY))
        if verbose:
            print("Count of estimators: " + str(i) + ", score: " + str(results[i][1]))
    return results

def get_results_gbdt(n_est, verbose=True, init=None):
    results = dict()
    for i in n_est:
        if init is not None:
            reg = GradientBoostingRegressor(n_estimators=i, random_state=1234, verbose=0, init=init)
        else:
            reg = GradientBoostingRegressor(n_estimators=i, random_state=1234, verbose=0)
        reg.fit(trainX, trainY)
#         results[i] = (mse(reg.predict(trainX), trainY), mse(reg.predict(testX), testY))
        predict = reg.predict(testX)
        results[i] = (acc(round_predict(reg.predict(trainX)), trainY), acc(round_predict(predict), testY))
        if verbose:
            print("Count of estimators: " + str(i) + ", score: " + str(results[i][1]))    
    return results

def get_optimal(results):
    results = sorted(results, key=lambda x: x[1])

## Bagging and RSM

Строим зависимость SMAPE на train и test от количества базовых алгоритмов. Количество базовых алгоритмов берется из диапазона (10, 200), как можно видеть на графиках ниже, ошибка на train в таком диапазоне стабилизируется.

In [15]:
bin_data = get_binarized_data(data, Cathegorical_columns)
trainX, testX, trainY, testY = split_train_test(df = bin_data, test_size=0.5)

### #1
**Bagging:** 0.8 от размера тестовой выборки.

**RSM:** 0.8 от размера тестовой выборки.

In [22]:
n_est = range(10, 150, 10)
%time results = get_results_baggind_rsm(n_est, 0.5, 0.75, verbose=True)
show_plot(n_est, results)

## Random Forest
Строим зависимость SMAPE на train и test от количества базовых алгоритмов. Количество базовых алгоритмов берется из диапазона (10, 200).

In [23]:
# n_est = range(10, 150, 10)
# %time results = get_results_rf(n_est)
# show_plot(n_est, results)

## Extremely Randomized Trees Ensemble
Строим зависимость SMAPE на train и test от количества базовых алгоритмов. Количество базовых алгоритмов берется из диапазона (10, 200).

In [25]:
# %time results = get_results_erte(n_est)
# show_plot(n_est, results)

## Gradient Boosted Decision Trees
Строим зависимость SMAPE на train и test от количества базовых алгоритмов. Количество базовых алгоритмов берется из диапазона (10, 200).

In [26]:
# %time results = get_results_gbdt(n_est)
# show_plot(n_est, results)

## Вывод
?

# Использование категориальных признаков

In [ ]:
##Gradient Boosted Decision Trees
## Random Forest
## Bagging and RSM
## Extreemly Randomize

## Вывод

# Другие модели и генерация дополнительных признаков

## Вывод